In [1]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Create embeddings and a vector db out of stored index
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Meilisearch

embeddings = OpenAIEmbeddings()
vectordb = Meilisearch(embedding=embeddings, index_name='la-sara-knowledge')

In [3]:
# Create a retriever from the vector db
retriever = vectordb.as_retriever()

query = "I want a free consultation."
retriever.get_relevant_documents(query=query)

[Document(page_content='for schedule free consultation visit https://calendly.com/d/grf-5sr-mtt this url', metadata={'source': 'booking_url.txt'}),
 Document(page_content='with the doctor. Testimonials', metadata={'source': 'about_page.txt'}),
 Document(page_content='first step towards restoring natural functionality by scheduling a visit with the doctor.', metadata={'source': 'about_page.txt'}),
 Document(page_content='Testimonials:', metadata={'source': 'home_page.txt'})]

In [17]:
# Create a conversation agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
conversational_memory = ConversationBufferWindowMemory(memory_key='chat_history', k=5, return_messages=True)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

In [18]:
# Test the retriever
qa.run(query)

'You can schedule a free consultation with the doctor by visiting this URL: https://calendly.com/d/grf-5sr-mtt.'

In [19]:
# Let's convert the retrieval chain into a tool
from langchain.agents import Tool

tools = [
    Tool(
        name='LaSara Knowledge Base', 
        func=qa.run, 
        description='use this tool when answering general knowledge queries to get more information about lasara medical group'
    )
]

In [20]:
# Now, we can initialize the agent
from langchain.agents import initialize_agent
from langchain.agents import AgentType

system_message = "You are an intelligent AI assistant for LaSara Medical Group. Provide necessary information about LaSara Medical Group that are asked to you. Look into your knowledge database for information. In case anything is not found, politely say that you don't know."

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
    early_stopping_method='generate',
    memory=conversational_memory,
    agent_kwargs={
             "system_message": system_message
            }
)


In [30]:
agent.run("hi i'm david. how can I book a consultancy?")



> Entering new AgentExecutor chain...
{
    "action": "LaSara Knowledge Base",
    "action_input": "how to book a consultancy"
}
Observation: To book a consultation, you can visit the following URL: https://calendly.com/d/grf-5sr-mtt. This will allow you to schedule a free consultation with the doctor.
Thought:{
    "action": "Final Answer",
    "action_input": "To book a consultancy with LaSara Medical Group, you can visit the following URL: [Book Consultancy](https://calendly.com/d/grf-5sr-mtt). This will allow you to schedule a free consultation with the doctor."
}

> Finished chain.


'To book a consultancy with LaSara Medical Group, you can visit the following URL: [Book Consultancy](https://calendly.com/d/grf-5sr-mtt). This will allow you to schedule a free consultation with the doctor.'

In [31]:
agent.run('What are the services of LaSara Medical Group?')



> Entering new AgentExecutor chain...
{
    "action": "LaSara Knowledge Base",
    "action_input": "services of LaSara Medical Group"
}
Observation: LaSara Medical Group offers a range of services to help patients regenerate and optimize their health. While I don't have specific information about the services they provide, you can visit their official website or contact them directly to get more details about the specific services they offer.
Thought:{
    "action": "Final Answer",
    "action_input": "LaSara Medical Group offers a range of services to help patients regenerate and optimize their health. For specific details about the services they provide, it is recommended to visit their official website or contact them directly."
}

> Finished chain.


'LaSara Medical Group offers a range of services to help patients regenerate and optimize their health. For specific details about the services they provide, it is recommended to visit their official website or contact them directly.'

In [32]:
agent.run('What was my name?')



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Your name is David"
}

> Finished chain.


'Your name is David'

In [33]:
agent.run('Give me physical address of LaSara')



> Entering new AgentExecutor chain...
{
    "action": "LaSara Knowledge Base",
    "action_input": "physical address"
}
Observation: The physical address is 8929 University Center Ln Ste 100, San Diego, CA 92122.
Thought:{
    "action": "Final Answer",
    "action_input": "The physical address of LaSara Medical Group is 8929 University Center Ln Ste 100, San Diego, CA 92122."
}

> Finished chain.


'The physical address of LaSara Medical Group is 8929 University Center Ln Ste 100, San Diego, CA 92122.'

In [34]:
agent.run('How many shockwave treatments had LaSara completed?')



> Entering new AgentExecutor chain...
{
    "action": "LaSara Knowledge Base",
    "action_input": "Number of shockwave treatments completed"
}
Observation: Over 11,000 shockwave treatments have been completed since 2017.
Thought:{
    "action": "Final Answer",
    "action_input": "Over 11,000 shockwave treatments have been completed since 2017."
}

> Finished chain.


'Over 11,000 shockwave treatments have been completed since 2017.'